In [1]:
import pandas as pd
import numpy as np
import torch
import gensim.downloader as api
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
words = api.load("glove-wiki-gigaword-50")
nltk.download('wordnet')

[==================================================] 100.0% 66.0/66.0MB downloaded


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
# import os
# os.chdir('/content/drive/MyDrive/NLP/Natural-Language-Processing-with-Disaster-Tweets-master/Codes')

In [15]:
data = pd.read_csv('data.csv')

In [7]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
lemmatizer = WordNetLemmatizer()

def make_tokens(s):
    s = s.lower()
    tokens = tokenizer.tokenize(s)
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return [t for t in tokens if t in words]

def wordVectors(s):
    lis = make_tokens(s)
    v = []

    for token in lis:
        
        if token not in words:
            continue
    
        if token not in t:
            t[token] = ind[0]
            w.append(words[token])
            ind[0] += 1
        
        v.append(np.array(t[token]).reshape(1))
        
    return np.array(v, dtype=float)

def padding(df, colName):
    sequences = []
    
    for i in range(df.shape[0]):
        seq = wordVectors(df[colName][i])
        
        if seq.shape[0] == 0:
            seq = np.zeros(shape=(1, 1))
        
        sequences.append(seq)
    
    cpy = [arr.copy() for arr in sequences]
    
    for i in range(len(sequences)):
        cpy[i] = np.concatenate([sequences[i], np.zeros(shape=(35-sequences[i].shape[0], 1))])
        
    return np.array(cpy).astype(int)

def makeDf(df,colName):
    return padding(df, colName)

t = {}
w = [np.zeros(50).astype(float)]
ind = [1]

X = makeDf(data.copy(),'tweet')
y = data['label'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

w = np.stack(w, axis=0)
w = torch.from_numpy(w).float()

In [8]:
class RNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, emb):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = torch.nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, num_classes)
        self.embed = torch.nn.Embedding.from_pretrained(emb)
        
    def forward(self, x):
        x = self.embed(x)[:,0,:,:]
        x = torch.transpose(x, 2, 1)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = out.mean(dim=1)
        out = self.fc(out)
        out = torch.squeeze(out,1)
        return out

In [9]:
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, emb, bidirectional=False):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=self.bidirectional)
        
        self.fc = None
        if self.bidirectional:
            self.fc = torch.nn.Linear(hidden_size*2, num_classes)
        else:
            self.fc = torch.nn.Linear(hidden_size, num_classes)
        
        self.embed = torch.nn.Embedding.from_pretrained(emb)
        
    def forward(self, x):
        x = self.embed(x)[:,0,:,:]
        x = torch.transpose(x, 2, 1)
        h0 = None
        c0 = None
        
        if self.bidirectional:
            h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
            c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        else:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0,c0))
        out = out.mean(dim=1)
        out = self.fc(out)
        out = torch.squeeze(out,1)
        return out

In [10]:
class DAN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, emb):
        super(DAN, self).__init__()
        self.layers = torch.nn.Sequential(torch.nn.Linear(input_size, hidden_size),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(hidden_size, num_classes))
        self.embed = torch.nn.Embedding.from_pretrained(emb)
    def forward(self, x):
        x = self.embed(x)[:,0,:,:]
        x = torch.transpose(x, 2, 1)
        x = x.mean(dim=1)
        x = self.layers(x)
        x = torch.squeeze(x,1)
        return x

In [11]:
#fixed
num_classes = 1
num_layers = 1
batch_size = 30
input_size = 35
sequence_length = 1

In [12]:
train_dataset = []
for i in range(len(X_train)):
    train_dataset.append([torch.from_numpy(X_train[i]), y_train[i]])
    
test_dataset = []
for i in range(len(X_test)):
    test_dataset.append(torch.from_numpy(X_test[i]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size)

In [13]:
dan_params = {'num_epochs':150, 'learning_rate':0.1, 'hidden_size':50}
rnn_params = {'num_epochs':25, 'learning_rate':0.1, 'hidden_size':50}
lstm_params = {'num_epochs':100, 'learning_rate':0.1, 'hidden_size':50}
bilstm_params = {'num_epochs':20, 'learning_rate':1, 'hidden_size':50}

In [14]:
for mod in ['RNN','LSTM','Bi-LSTM','DAN']:
    print(mod)
    model = None
    if mod == 'RNN':
        num_epochs = rnn_params['num_epochs']
        learning_rate = rnn_params['learning_rate']
        hidden_size = rnn_params['hidden_size']
        model = RNN(input_size, hidden_size, num_layers, num_classes, w).to(device)
    if mod == 'LSTM':
        num_epochs = lstm_params['num_epochs']
        learning_rate = lstm_params['learning_rate']
        hidden_size = lstm_params['hidden_size']
        model = LSTM(input_size, hidden_size, num_layers, num_classes, w, bidirectional=False).to(device)
    if mod == 'Bi-LSTM':
        num_epochs = bilstm_params['num_epochs']
        learning_rate = bilstm_params['learning_rate']
        hidden_size = bilstm_params['hidden_size']
        model = LSTM(input_size, hidden_size, num_layers, num_classes, w, bidirectional=True).to(device)
    if mod == 'DAN':
        num_epochs = dan_params['num_epochs']
        learning_rate = dan_params['learning_rate']
        hidden_size = dan_params['hidden_size']
        model = DAN(input_size, hidden_size, num_classes, w).to(device)
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_values = []
    n_total_steps = len(train_loader)
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inp,labels) in enumerate(train_loader):
            inp = inp.reshape(-1, sequence_length, input_size).to(device)
            labels = labels.to(device)
          
            outputs = model(inp)
            loss = criterion(outputs, labels.float())
          
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
          
          #if (i+1) % 50 == 0:
          #    print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
          
            running_loss += loss.item() * inp.size(0)
          
        loss_values.append(running_loss / len(train_dataset))

    y_pred = []
    y_true = y_train.tolist()

    with torch.no_grad():
        for (inp,labels) in train_loader:
            inp = inp.reshape(-1, sequence_length, input_size).to(device)
              
            outputs = model(inp)
            predicted = torch.where(outputs >= 0.0, 1, 0)
            y_pred.extend(predicted.tolist())

    print('Train Accuracy:', accuracy_score(y_train, y_pred))

    y_pred = []
    y_true = y_test.tolist()

    with torch.no_grad():
        for (inp) in test_loader:
            inp = inp.reshape(-1, sequence_length, input_size).to(device)
              
            outputs = model(inp)
            predicted = torch.where(outputs >= 0.0, 1, 0)
            y_pred.extend(predicted.tolist())

    print('Test Accuracy:', accuracy_score(y_test, y_pred))
    print('Classification Report:')
    print(classification_report(y_test, y_pred))

RNN
Train Accuracy: 0.7326765188834155
Test Accuracy: 0.7367038739330269
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.85      0.79       874
           1       0.74      0.59      0.66       649

    accuracy                           0.74      1523
   macro avg       0.74      0.72      0.72      1523
weighted avg       0.74      0.74      0.73      1523

LSTM
Train Accuracy: 0.819047619047619
Test Accuracy: 0.7150361129349967
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.74      0.75       874
           1       0.66      0.67      0.67       649

    accuracy                           0.72      1523
   macro avg       0.71      0.71      0.71      1523
weighted avg       0.72      0.72      0.72      1523

Bi-LSTM
Train Accuracy: 0.7479474548440066
Test Accuracy: 0.7150361129349967
Classification Report:
              precision    recall  f1-score   support


In [16]:
#SVM
print('SVM:')
tfidf = TfidfVectorizer()
classifier = LinearSVC(tol=0.01,random_state = 42)
X = data['tweet']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
tfidf.fit(X)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)
classifier.fit(X_train,y_train )
y_pred = classifier.predict(X_train)
print('Train Accuracy:', accuracy_score(y_train, y_pred))
y_pred = classifier.predict(X_test)
print('Test Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))

SVM:
Train Accuracy: 0.9775041050903119
Test Accuracy: 0.7977675640183848
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       874
           1       0.80      0.70      0.75       649

    accuracy                           0.80      1523
   macro avg       0.80      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523

